# Step 3: Perform fit

In [ ]:
import pycompwa.ui as pwa

In [ ]:
particle_list = pwa.read_particles('model.xml')
kinematics = pwa.create_helicity_kinematics('model.xml', particle_list)
kinematics.create_all_subsystems()
data_sample = pwa.read_root_data(input_file='generated_data.root')
phsp_sample = pwa.read_root_data(input_file='generated_phsp.root')
intensity_builder = pwa.IntensityBuilderXML(
    'model.xml', particle_list, kinematics, phsp_sample)
intensity = intensity_builder.create_intensity()

## 3.1 Define estimator

To perform a fit, you need to define an **estimator**. An estimator is a measure for the discrepancy between the intensity model and the data distribution to which you fit. To create an estimator we therefore generally need:

The data sample of momentum tuples can be converted to a `DataSet` as follows:

In [ ]:
data_set = kinematics.convert(data_sample)

Now we can create an estimator. In this example, we use an **unbinned log likelihood** estimator. This estimator is used most commonly in high-energy physics.

In [ ]:
estimator, initial_parameters = \
    pwa.create_unbinned_log_likelihood_function_tree_estimator(
        intensity, data_set)

Notice that you not only receive a estimator object, but also a list of fit parameters. You use this list of fit parameters to initialize the optimizer later on.

The parameters contains the following info:
- initial value
- whether they are free or fixed
- optional upper and lower limits
- errors that give certain optimizers hints on the step size

In [ ]:
initial_parameters

## 3.2 Modify parameters

The fit parameters in this list have been initialized with the values stated in the XML file or with default values if unspecified, but they can easily be modified in Python as well:

In [ ]:
print("\nthis parameter is initially not fixed:")
print(initial_parameters[8])

initial_parameters[7].is_fixed = True
initial_parameters[8].is_fixed = True
print("\nand now it is fixed:")
print(initial_parameters[8])

To make the fit process a bit more interesting, we modify one of the parameters to a different initial value than the true value:

In [ ]:
original_value = initial_parameters[12].value
initial_parameters[12]

In [ ]:
initial_parameters[12].value = 2.0
initial_parameters[12]

## 3.3 Optimize the intensity model

Now it's time to perform the fit! This is quite simple: just create an optimizer instance of your choice, here [Minuit2](https://root.cern.ch/doc/master/Minuit2Page.html), and call its `optimize` method to start the fitting process.

The computation time depends on the complexity of the mode, number of data events, the size of the phase space sample, and the number of free parameters.

In [ ]:
print('data size', len(data_sample.events))
print('phsp size', len(phsp_sample.events))
print('free parameters:',
      len([par for par in initial_parameters if not par.is_fixed]))

In [ ]:
optimizer = pwa.MinuitIF()
fit_result = optimizer.optimize(estimator, initial_parameters)

In [ ]:
fit_result.fit_duration_in_seconds

Let's check whether the fit parameters are close to the true values.

In [ ]:
value = fit_result.final_parameters[12].value
error = fit_result.final_parameters[12].error
if abs(value - original_value) < 3 * error[0]:
    print('This value is close to', original_value, 'again:')
    print(value, "+-", error[0])  # upper and lower error are equal in hesse
else:
    raise Exception('Value does not lie within 3 sigma!')

In [ ]:
intensity.updateParametersFrom(fit_result.final_parameters)

## 3.4 Exporting and importing

Optimizing an intensity model can take a long time, so it is important that you store the fit result in the end. We again write to an XML file (but note that this has a different format than that of the amplitude model).

In [ ]:
fit_result.write('fit_result.xml')

In [ ]:
imported_fit_result = pwa.load('fit_result.xml')

exported_value = fit_result.final_parameters[12].value
imported_value = imported_fit_result.final_parameters[12].value

if exported_value != imported_value:
    raise Exception
else:
    print(imported_fit_result.final_parameters[12])